### CSC427 Quantum Day Three

_Date: April 26, 2022_

_Author: Burton Rosenberg_


References, 

- https://qiskit.org/textbook/ch-algorithms/grover.html#3qubits
- C. Figgatt, D. Maslov, K. A. Landsman, N. M. Linke, S. Debnath & C. Monroe (2017), "Complete 3-Qubit Grover search on a programmable quantum computer", Nature Communications, Vol 8, Art 1918, doi:10.1038/s41467-017-01904-7, arXiv:1703.10535 
- https://arxiv.org/abs/1703.10535
- https://quantum-computing.ibm.com/composer/docs/iqx/guide/advanced-single-qubit-gates



In [1]:
import qiskit
import time, math

from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.jobstatus import JOB_FINAL_STATES, JobStatus
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

qiskit.__qiskit_version__

#initialization
import matplotlib.pyplot as plt
import numpy as np

# what is this?
from qiskit.providers.ibmq import least_busy

args_g = []

# your api token from IBM, first time run.
# after that None is good

#api_token = 'abcdefghijklmnopqrstuvwxyz'
api_token = None 

def load_or_save_IBMQ_account(api_token=None):
    global args_g
    print('getting provider...')
    if api_token:
        # only needs to be done once
        # then is stored in e.g. ~/.qistkit/qiskitrc
        IBMQ.save_account(api_token)
    provider = IBMQ.load_account()
    return provider

def list_backends(provider):
    global args_g
    print('backends available ...')
    backends = provider.backends()
    for be in backends:
        st = be.status()
        if st.operational:
            print(f'\t{be.name()}, pending jobs:{st.pending_jobs}')

            
def run_quantum_circuit_on_backend(quantum_circuit,provider,backend):
    backend = provider.get_backend(backend)
    qobj = assemble(transpile(quantum_circuit, backend=backend), backend=backend)
    job = backend.run(qobj)
    return job


def wait_for_job(backend, job, wait_interval=5):
    backend = provider.get_backend(backend)
    retrieved_job = backend.retrieve_job(job.job_id())
    start_time = time.time()
    job_status = job.status()
    while job_status not in JOB_FINAL_STATES:
        print(f'Status @ {time.time() - start_time:0.0f} s: {job_status.name},'
              f' est. queue position: {job.queue_position()}')
        time.sleep(wait_interval)
        job_status = job.status()


provider = load_or_save_IBMQ_account(api_token)
list_backends(provider)

# choose your backend

backend = 'ibmq_qasm_simulator'
#backend = 'ibmq_armonk'
#backend = 'ibmq_vigo'
#backend = 'ibmq_london'
#backend = 'ibmq_lima'

# and so forth ... chose from the results given by provider.backends()

getting provider...
backends available ...
	ibmq_qasm_simulator, pending jobs:3
	ibmq_armonk, pending jobs:5
	ibmq_santiago, pending jobs:51
	ibmq_bogota, pending jobs:72
	ibmq_lima, pending jobs:1
	ibmq_belem, pending jobs:4
	ibmq_quito, pending jobs:13
	simulator_statevector, pending jobs:3
	simulator_mps, pending jobs:3
	simulator_extended_stabilizer, pending jobs:3
	simulator_stabilizer, pending jobs:3
	ibmq_manila, pending jobs:29


### Controlled Hadamard gate

In [2]:
import math 


def add_controlled_hadamard(circuit):
    circuit.crx(math.pi/4,0,2)
    circuit.crx(math.pi/4,1,2)
    circuit.cnot(0,1)
    circuit.crx(-math.pi/4,1,2)
    circuit.cnot(0,1)
    


def test_2_input():
    ch_a = []
    for i in range(4):
        ch = QuantumCircuit(3)
        if i%2==1:
            ch.x(0)
        if i>1:
            ch.x(1)
        add_controlled_hadamard(ch)
        ch.measure_all()
        ch_a += [ch]
    return ch_a
  
ch_a = test_2_input()
for ch in ch_a:
    print(ch.draw())
    

                                                     ░ ┌─┐      
   q_0: ─────■──────────────────■────────────────■───░─┤M├──────
             │                ┌─┴─┐            ┌─┴─┐ ░ └╥┘┌─┐   
   q_1: ─────┼──────────■─────┤ X ├─────■──────┤ X ├─░──╫─┤M├───
        ┌────┴────┐┌────┴────┐└───┘┌────┴─────┐└───┘ ░  ║ └╥┘┌─┐
   q_2: ┤ Rx(π/4) ├┤ Rx(π/4) ├─────┤ Rx(-π/4) ├──────░──╫──╫─┤M├
        └─────────┘└─────────┘     └──────────┘      ░  ║  ║ └╥┘
meas: 3/════════════════════════════════════════════════╩══╩══╩═
                                                        0  1  2 
        ┌───┐                                             ░ ┌─┐      
   q_0: ┤ X ├─────■──────────────────■────────────────■───░─┤M├──────
        └───┘     │                ┌─┴─┐            ┌─┴─┐ ░ └╥┘┌─┐   
   q_1: ──────────┼──────────■─────┤ X ├─────■──────┤ X ├─░──╫─┤M├───
             ┌────┴────┐┌────┴────┐└───┘┌────┴─────┐└───┘ ░  ║ └╥┘┌─┐
   q_2: ─────┤ Rx(π/4) ├┤ Rx(π/4) ├─────┤ Rx(-π/4) ├──────░──╫──╫

In [3]:
aer_sim = Aer.get_backend('aer_simulator')

for ch in ch_a:
    qobj = assemble(transpile(ch, aer_sim))
    results = aer_sim.run(qobj).result()
    counts = results.get_counts()
    print(f'results: {results.get_counts()}')
    plot_histogram(counts)



results: {'000': 1024}
results: {'001': 1024}
results: {'010': 1024}
results: {'011': 546, '111': 478}


### Toffoli gate

As in a controlled NOT; and using Z transforms

In [4]:
import math 


def add_toffoli(circuit):
    circuit.barrier()
    circuit.h(2)
    circuit.crz(math.pi/2,0,2)
    circuit.crz(math.pi/2,1,2)
    circuit.cnot(0,1)
    circuit.crz(-math.pi/2,1,2)
    circuit.h(2)
    circuit.cnot(0,1)
    circuit.barrier()

def test_2_input():
    ch_a = []
    for i in range(4):
        ch = QuantumCircuit(3)
        if i%2==1:
            ch.x(0)
        if i>1:
            ch.x(1)
        add_toffoli(ch)
        ch.measure_all()
        ch_a += [ch]
    return ch_a


ch_a = test_2_input()
for ch in ch_a:
    print(ch.draw(output='text'))
    #ch.draw(output='mpl')


         ░                                                   ░  ░ ┌─┐      
   q_0: ─░───────────■──────────────────■────────────────■───░──░─┤M├──────
         ░           │                ┌─┴─┐            ┌─┴─┐ ░  ░ └╥┘┌─┐   
   q_1: ─░───────────┼──────────■─────┤ X ├─────■──────┤ X ├─░──░──╫─┤M├───
         ░ ┌───┐┌────┴────┐┌────┴────┐└───┘┌────┴─────┐├───┤ ░  ░  ║ └╥┘┌─┐
   q_2: ─░─┤ H ├┤ Rz(π/2) ├┤ Rz(π/2) ├─────┤ Rz(-π/2) ├┤ H ├─░──░──╫──╫─┤M├
         ░ └───┘└─────────┘└─────────┘     └──────────┘└───┘ ░  ░  ║  ║ └╥┘
meas: 3/═══════════════════════════════════════════════════════════╩══╩══╩═
                                                                   0  1  2 
        ┌───┐ ░                                                   ░  ░ ┌─┐   »
   q_0: ┤ X ├─░───────────■──────────────────■────────────────■───░──░─┤M├───»
        └───┘ ░           │                ┌─┴─┐            ┌─┴─┐ ░  ░ └╥┘┌─┐»
   q_1: ──────░───────────┼──────────■─────┤ X ├─────■──────┤ X ├─░──░──╫─┤M├»


In [5]:

aer_sim = Aer.get_backend('aer_simulator')
for ch in ch_a:
    qobj = assemble(transpile(ch, aer_sim))
    results = aer_sim.run(qobj).result()
    counts = results.get_counts()
    print(f'results: {results.get_counts()}')
    plot_histogram(counts)



results: {'000': 1024}
results: {'001': 1024}
results: {'010': 1024}
results: {'111': 1024}


### swapping two q-bits using c-nots

In [6]:


def add_swap_circuit(circuit):
    circuit.cx(0, 1)
    circuit.cx(1, 0)
    circuit.cx(0, 1)
    return circuit
    
def test_2_input():
    ch_a = []
    for i in range(4):
        ch = QuantumCircuit(3)
        if i%2==1:
            ch.x(0)
        if i>1:
            ch.x(1)
        add_swap_circuit(ch)
        ch.measure_all()
        ch_a += [ch]
    return ch_a
  
ch_a = test_2_input()

aer_sim = Aer.get_backend('aer_simulator')

for ch in ch_a:
    qobj = assemble(transpile(ch, aer_sim))
    results = aer_sim.run(qobj).result()
    counts = results.get_counts()
    print(ch.draw())
    print(f'results: {results.get_counts()}')
    plot_histogram(counts)


             ┌───┐      ░ ┌─┐      
   q_0: ──■──┤ X ├──■───░─┤M├──────
        ┌─┴─┐└─┬─┘┌─┴─┐ ░ └╥┘┌─┐   
   q_1: ┤ X ├──■──┤ X ├─░──╫─┤M├───
        └───┘     └───┘ ░  ║ └╥┘┌─┐
   q_2: ────────────────░──╫──╫─┤M├
                        ░  ║  ║ └╥┘
meas: 3/═══════════════════╩══╩══╩═
                           0  1  2 
results: {'000': 1024}
        ┌───┐     ┌───┐      ░ ┌─┐      
   q_0: ┤ X ├──■──┤ X ├──■───░─┤M├──────
        └───┘┌─┴─┐└─┬─┘┌─┴─┐ ░ └╥┘┌─┐   
   q_1: ─────┤ X ├──■──┤ X ├─░──╫─┤M├───
             └───┘     └───┘ ░  ║ └╥┘┌─┐
   q_2: ─────────────────────░──╫──╫─┤M├
                             ░  ║  ║ └╥┘
meas: 3/════════════════════════╩══╩══╩═
                                0  1  2 
results: {'010': 1024}
                  ┌───┐      ░ ┌─┐      
   q_0: ───────■──┤ X ├──■───░─┤M├──────
        ┌───┐┌─┴─┐└─┬─┘┌─┴─┐ ░ └╥┘┌─┐   
   q_1: ┤ X ├┤ X ├──■──┤ X ├─░──╫─┤M├───
        └───┘└───┘     └───┘ ░  ║ └╥┘┌─┐
   q_2: ─────────────────────░──╫──╫─┤M├
               